<a href="https://colab.research.google.com/github.com/greatestgoat/kaggle-template/tree/PushNotebooks/project/Notebook/templateColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

/bin/bash: nvidia-smi: command not found


# Configurations

In [24]:
import os
from dotenv import load_dotenv
from pathlib import Path
env_path = Path('..') / '.env'
load_dotenv(dotenv_path=env_path)

try:
    from google.colab import auth
    auth.authenticate_user()
except ImportError:
    pass

GITHUB_OWNER = os.getenv('GITHUB_OWNER')
REPOSITORY_NAME = os.getenv("REPOSITORY_NAME")
PROJECT_ID = os.getenv("PROJECT_ID")
BUCKET_NAME = os.getenv("BUCKET_NAME")

os.environ["GCLOUD_PROJECT"] = PROJECT_ID

In [22]:
os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

'./conn-gcs.json'

In [23]:
from google.cloud import storage
storage_client = storage.Client(project=PROJECT_ID)

In [ ]:
!git clone https://github.com/{GITHUB_OWNER}/{REPOSITORY_NAME}.git

In [ ]:
!pip install -qr ./{REPOSITORY_NAME}/requirements.txt

In [ ]:
cd ./{REPOSIZETORY_NAME}/project/src

In [ ]:
!gcloud config set project {PROJECT_ID}

In [ ]:
blobs = storage_client.list_blobs(BUCKET_NAME)
for blob in blobs:
    filepath = blob.name
    if filepath[:7]=="mlruns/" and len(filepath)!=7:
        os.makedirs("/".join(filepath.split("/")[:-1]), exist_ok=True)
        blob.download_to_filename(filepath)

# Run script

In [ ]:
!python helloworld.py

# Save file to GCS(mlruns)

In [ ]:
import glob

files = [f for f in glob.glob("mlruns/**", recursive=True) if f[:7]=="mlruns/" and len(f)!=7]
files_in_bucket = [f.name for f in storage_client.list_blobs(BUCKET_NAME)]
bucket = storage_client.get_bucket(BUCKET_NAME)

for f in files:
    try:
        if f not in files_in_bucket:
            blob = bucket.blob(f)
            blob.upload_from_filename(f)
    except:
        pass